In [11]:
! docker-compose up -d

Creating network "full-stack_default" with the default driver
Creating full-stack_mids_1 ... 
Creating full-stack_cloudera_1 ... 
Creating full-stack_zookeeper_1 ... 
Creating full-stack_spark_1     ... mdone
Creating full-stack_kafka_1     ... mdone
ting full-stack_kafka_1     ... done

In [12]:
! docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Created topic events.


In [32]:
# run this in a new bash terminal 
# I modified the game_api file to take in the new data format
# game_api2.py

! docker-compose exec mids env FLASK_APP=/w205/full-stack/game_api2.py flask run --host 0.0.0.0

 * Serving Flask app "game_api"
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
^C


In [33]:
# run this in a second new bash terminal

! docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning

^C


In [109]:
df.size

1100

In [30]:
from pandas import DataFrame, read_csv
import pandas as pd 

file = r'sample_data_v3.csv'
df = pd.read_csv(file)

#for i in range(0,df.shape[0]): # for the entire data set
for i in range(0,1): # for testing
    
        id = str(df.iloc[i, 1])
        month = str(df.iloc[i, 2])
        hrs = str(df.iloc[i, 3])
        ref = str(df.iloc[i, 4])
        sub = str(df.iloc[i, 5])
        money = str(df.iloc[i, 6])
        acctopen = str(df.iloc[i, 7])
        sword = str(df.iloc[i, 8])
        guild = str(df.iloc[i, 9])
        swag = str(df.iloc[i, 10])
        
        # the host name is composed of user id and month
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/play_hrs/"$hrs"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/ref_count/"$ref"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/sub_count/"$sub"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/money_paid/"$money"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/account_open/"$acctopen"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/purchase_sword/"$sword"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/join_guild/"$guild"
        ! docker-compose exec mids ab -n 1 -H "Host: $id $month" http://localhost:5000/sw_a_g/"$swag"
    
    

This is ApacheBench, Version 2.3 <$Revision: 1706008 $>
Copyright 1996 Adam Twiss, Zeus Technology Ltd, http://www.zeustech.net/
Licensed to The Apache Software Foundation, http://www.apache.org/

Benchmarking localhost (be patient).....done


Server Software:        Werkzeug/0.14.1
Server Hostname:        localhost
Server Port:            5000

Document Path:          /play_hrs/11.76019947
Document Length:        14 bytes

Concurrency Level:      1
Time taken for tests:   0.003 seconds
Complete requests:      1
Failed requests:        0
Total transferred:      169 bytes
HTML transferred:       14 bytes
Requests per second:    291.89 [#/sec] (mean)
Time per request:       3.426 [ms] (mean)
Time per request:       3.426 [ms] (mean, across all concurrent requests)
Transfer rate:          48.17 [Kbytes/sec] received

Connection Times (ms)
              min  mean[+/-sd] median   max
Connect:        0    0   0.0      0       0
Processing:     3    3   0.0      3       3
Waiting:        3   

In [31]:
! docker-compose exec spark spark-submit /w205/full-stack/filtered_writes2.py

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/04/06 03:42:20 INFO SparkContext: Running Spark version 2.2.0
21/04/06 03:42:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/04/06 03:42:21 INFO SparkContext: Submitted application: ExtractEventsJob
21/04/06 03:42:21 INFO SecurityManager: Changing view acls to: root
21/04/06 03:42:21 INFO SecurityManager: Changing modify acls to: root
21/04/06 03:42:21 INFO SecurityManager: Changing view acls groups to: 
21/04/06 03:42:21 INFO SecurityManager: Changing modify acls groups to: 
21/04/06 03:42:21 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(root); groups with view permissions: Set(); users  with modify permissions: Set(root); groups with modify permissions: Set()
21/04/06 03:42:21 INFO Utils: Successfully started service 'sparkDriver' on port 43327.
21/04/

In [32]:
! docker-compose exec cloudera hadoop fs -ls /tmp/play_hrs/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:42 /tmp/play_hrs/_SUCCESS
-rw-r--r--   1 root supergroup       1809 2021-04-06 03:42 /tmp/play_hrs/part-00000-6489e950-fa3d-4a84-b045-d43d71bb5fdf-c000.snappy.parquet


In [33]:
! docker-compose exec cloudera hadoop fs -ls /tmp/ref_count/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:42 /tmp/ref_count/_SUCCESS
-rw-r--r--   1 root supergroup       1746 2021-04-06 03:42 /tmp/ref_count/part-00000-52638758-a648-4c27-aec6-7717bc255806-c000.snappy.parquet


In [34]:
! docker-compose exec cloudera hadoop fs -ls /tmp/purchase_sword/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:42 /tmp/purchase_sword/_SUCCESS
-rw-r--r--   1 root supergroup       1723 2021-04-06 03:42 /tmp/purchase_sword/part-00000-cef46c4a-7b40-4213-b3d3-8a663b32edb5-c000.snappy.parquet


In [35]:
! docker-compose exec cloudera hadoop fs -ls /tmp/sw_a_g/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:43 /tmp/sw_a_g/_SUCCESS
-rw-r--r--   1 root supergroup       1704 2021-04-06 03:43 /tmp/sw_a_g/part-00000-ef28ceec-aedf-4a40-94af-ad5f52b003b0-c000.snappy.parquet


In [36]:
! docker-compose exec cloudera hadoop fs -ls /tmp/join_guild/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:42 /tmp/join_guild/_SUCCESS
-rw-r--r--   1 root supergroup       1718 2021-04-06 03:42 /tmp/join_guild/part-00000-6ca368c9-4e2c-4c58-896e-26dad9ba2522-c000.snappy.parquet


In [37]:
! docker-compose exec cloudera hadoop fs -ls /tmp/account_open/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:42 /tmp/account_open/_SUCCESS
-rw-r--r--   1 root supergroup       1729 2021-04-06 03:42 /tmp/account_open/part-00000-2dba2735-2798-49ea-afb9-247a6e4b71f9-c000.snappy.parquet


In [38]:
! docker-compose exec cloudera hadoop fs -ls /tmp/money_paid/

Found 2 items
-rw-r--r--   1 root supergroup          0 2021-04-06 03:42 /tmp/money_paid/_SUCCESS
-rw-r--r--   1 root supergroup       1847 2021-04-06 03:42 /tmp/money_paid/part-00000-f4da7c3c-bcca-40cf-a048-f9ca3f65d24b-c000.snappy.parquet


In [8]:
! docker-compose down

Stopping full-stack_kafka_1     ... 
Stopping full-stack_spark_1     ... 
Stopping full-stack_cloudera_1  ... 
Stopping full-stack_mids_1      ... 
Stopping full-stack_zookeeper_1 ... 
Removing full-stack_kafka_1     ... mdone
Removing full-stack_spark_1     ... 
Removing full-stack_cloudera_1  ... 
Removing full-stack_mids_1      ... 
Removing full-stack_zookeeper_1 ... 
Removing network full-stack_default2mdone
